In [ ]:
import nltk
from nltk.tag import pos_tag
import pandas as pd
import numpy as np
import spacy
import re
import calendar
import os
import time
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from datetime import datetime
from typing_extensions import Literal
import spacy
from pii_codex.services.analysis_service import PIIAnalysisService
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.chunk import ne_chunk
from nltk import pos_tag
import warnings
from openpyxl import load_workbook
import logging

# Suppress all warnings
warnings.filterwarnings("ignore")

pii_analysis_service = PIIAnalysisService() # Install PII_Codex package following instrucions on Github: https://github.com/EdyVision/pii-codex?tab=readme-ov-file

# Load SpaCy's English NER model
nlp=spacy.load("en_core_web_sm")  # Spacy's trained pipeline package, can be downloaded python -m spacy download en_core_web_sm

# Set up logging configuration to suppress NLTK download messages
logging.getLogger('nltk').setLevel(logging.ERROR)

def download_missing_packages():
    # List of required packages and their identifiers
    packages = {
        'averaged_perceptron_tagger': 'taggers/averaged_perceptron_tagger',
        'maxent_ne_chunker': 'chunkers/maxent_ne_chunker',
        'words': 'corpora/words',
        'punkt': 'tokenizers/punkt'
    }
    
    for package, identifier in packages.items():
        try:
            nltk.data.find(identifier)
            print(f"Package '{package}' is already installed.")
        except LookupError:
            print(f"Package '{package}' not found. Downloading...")
            nltk.download(package)
            print(f"Package '{package}' has been downloaded and installed.")

# Run the function to check and download missing packages
download_missing_packages()


In [27]:
#df_Scrubbed= pd.read_csv("Major Safety Events_Scrubbed.csv") # load data
df= pd.read_excel("SS Major Events 2014 to present - Dossier - 2024-08-01T135318.165.xlsx") # load data


In [ ]:
df2= df.copy()
df2.shape

In [33]:
def scrub_vehicle_numbers(text):
    def redact_numbers(match):
        # Check if the match starts with a vehicle type (like "Coach") followed by numbers
        if re.match(r'\b((?:Bus|Train|Coach|Car|Vehicle|MV|VIN|Cars|Lead|LRV)\s*)\d+', match.group(), re.IGNORECASE):
            # Redact only the numbers, preserving the vehicle type
            return match.group(1) + "<REDACTED>"
        else:
            return match.group()  # Return original match if not following a vehicle type
    def redact_hyphen_numbers(match):
        # Check if the match is numbers separated by hyphens and not in MM-DD-YYYY format
        if re.match(r'\b\d+-\d+\b', match.group()) and not re.match(r'\b\d{1,2}-\d{1,2}-\d{4}\b', match.group()):
            return '<REDACTED>'
        else:
            return match.group()  # Return original match
        
        
    patterns = [
        (r'\b((?:Bus|Train|Coach|Car|Vehicle|MV|VIN|Cars|Lead|cab)\s*)\d+\b', redact_numbers),  # Redact numbers following vehicle types
        (r'(?<=#)\d+', "<REDACTED>"),        # Redact numbers following '#'
        (r'#:\s*(\d+)', lambda match: '#: <REDACTED>'),
        (r'#\s*(?:\(\d+\)|\d+)', "<REDACTED>"), 
        (r'\b\d+@\S+\b', "<REDACTED>"),     # Redact patterns like "03202022 @example" 
        (r"SITS\s*#?\s*\d+", "<REDACTED>"),  # Redact digits following SITS 
        (r'LRV\s*#?\s*\d+(?:-\d+)*', "<REDACTED>"),  # Redact LRV licence plate
        (r'\b\d+-\d+(?:-\d+)*\b', redact_hyphen_numbers),  # Redact numbers separated by hyphens if not in date format
        (r'\b(?:[A-Z]+(?:-[A-Z]*)*\d+(?:-\d+)*)\b', "<REDACTED>" ),
        (r'\bUNIT\s*#?\d+|\bUnit\s*#?\d+', "<REDACTED>"),
        (r'\btrain\s+consist[^\d]*(\d+(?:\s+\d+)*)(?!\d)', ' train consist <REDACTED>' ),
        (r'\([^\)]*[0-9a-zA-Z][^\)]*\)',"<REDACTED>"),
        (r"(?i)\bDOB\s*[\d\/\-\.\s]{7,}|\bDOB[\d\/\-\.\s]{7,}",  "DOB <REDACTED>"), # Redact DOB
        (r'(?i)license\s*plate\s*(?:\(?\s*([^\)\s]+)\s*\)?)', "license plate <REDACTED>"), # Redact license plate
        (r'\b\d{3}\b', ""),
       
        
    ]
    
    # Apply redaction patterns to the entire text
    for pattern, replacement in patterns:
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)   
    
    return text
    
    
#Function to scrub PII after occupation roles
def scrub_names_after_occupation_title(text):
    # Define regular expression patterns
    name_pattern_7= r'\b[A-Z]\.(?: [A-Z][a-zA-Z\-]+)+\b' #pattern to match names formatted as initials followed by one or more last names
    name_pattern_1 = r'\b(?:Conductor|Operator|police\s+officer)\s+(?!Assault(?:ed)?|Stated?)\b[A-Z][a-zA-Z]*\b(?:\s+[A-Z][a-zA-Z]*)*' # Pattern to match titles like Conductor or Operator followed by a name
    name_pattern_2 =  r'\b(OPERATOR|CONDUCTOR|engineer|officer|superintendent (transportation|of transportation)|track (walker|foreman)|supervisor|chief officer (of operations|maintainance of way|superintendent (of (transportation|infrastructure|maintenance of way)|system safety|infrastructure)|deputy superintendent))\b(?!\s+(?:\b(?:WAS|IS|HAS|HAD)\b|\w+(?:ED|ING)))\s+([A-Z][a-zA-Z]*\b(?:\s+[A-Z][a-zA-Z]*)*)'
    name_pattern_3=  r'Conductor\s*\((?!ASSAULT(?:ED|ING)?)[^)]*\)' #Pattern to match Conductor followed by a name in parentheses with exceptions : Assault, Assaulted
    name_pattern_4 =  r'Operator\s*\((?!ASSAULT(?:ED|ING)?)[^)]*\)'#Pattern to match Conductor followed by a name in parentheses with exceptions : Assault, Assaulted
    name_pattern_5 = r'\b(?:Det|Nurse|Police\s+Officer|P\.O\.|Sergeant|Sgt\.|OP\.|CON\.|Ofc\.|SGT|SGT\.|OP|PO|Det\.|police\s+officer|Mgr\.|Mgr|Mntr\.)\s+\b[A-Z][a-zA-Z]*\b(?:\s+[A-Z][a-zA-Z]*)*'## Pattern to match various titles followed by a name (e.g., Det, Nurse, Sgt, Police Officer, P.O., Sergeant)
    name_pattern_6 = r'(?:Dispatcher|Driver|Supervisor|Planner|Scheduler|Agent|Inspector|Officer|Captain|Deckhand|TM|CS|ATM|SME|Sgt|Detective|Capt\.|Lt\.|Sgt\.|detective|cs|Engineer|superintendent|Superintendent)\s+\b[A-Z][A-Za-z-]*\b(?:\s+[A-Z][A-Za-z-]*)*' # # Pattern to match transit occupation title followed by a name
    name_pattern_8= r'(?i)(PO\.|CON\.|Ofc\.|Lt|Lt\.|Sgt\.)(\s+[A-Z][a-zA-Z]*\s+[A-Z][a-zA-Z]*)'# 
    name_pattern_9= r'by\s[A-Z][A-Z\-\'\s]+(?:,\s[A-Z][A-Z\-\'\s]+)*'
        
    
    # Redact sensitive information
    text = re.sub(name_pattern_7, ' <REDACTED> ', text)
    text = re.sub(name_pattern_1, lambda match: match.group(0).split()[0] + ' <REDACTED>', text)  
    text = re.sub(name_pattern_2, lambda match: match.group(1) + ' <REDACTED>', text )
    text = re.sub(name_pattern_3, 'Conductor <REDACTED>', text, flags=re.IGNORECASE)
    text = re.sub(name_pattern_4, 'Operator <REDACTED>', text, flags=re.IGNORECASE)
    text = re.sub(name_pattern_5, lambda match: match.group(0).split()[0] + ' <REDACTED>', text)
    text = re.sub(name_pattern_6, lambda match: match.group(0).split()[0] + ' <REDACTED>', text)
    text = re.sub(name_pattern_8, r'\1 <REDACTED> <REDACTED>', text)
    text = re.sub(name_pattern_9, "[REDACTED]", text)

    return text


# Function to handle PII Codex exceptions 
def handle_pii_exception(text):
    doc = nlp(text)
    scrubbed_tokens = []

    for token in doc:
        if token.ent_type_ in ["PERSON", "LOC" ]:  #"GPE", "ORG", "MONEY", "QUANTITY", "EVENT", "FAC", "NORP"
            scrubbed_tokens.append("<REDACTED>")
        else:
            scrubbed_tokens.append(token.text)

    scrubbed_text = ' '.join(scrubbed_tokens)
    

    
    return scrubbed_text

def normalize_text(text):
    # Define a function to normalize a single line of text
    def normalize_line(line):
        tokens = nltk.word_tokenize(line)
        tagged = pos_tag(tokens)
        named_entities = ne_chunk(tagged)

        normalized_line = []
        for subtree in named_entities:
            if isinstance(subtree, nltk.Tree):  # Named entity
                # Preserve case for named entities
                normalized_line.append(' '.join(word for word, tag in subtree.leaves()))
            else:
                word, tag = subtree
                if word.isupper() and not re.match(r'^[A-Z]\.', word):  # Skip lowercase conversion for initials
                    normalized_line.append(word.lower())
                else:
                    normalized_line.append(word)

        normalized_line_str = ' '.join(normalized_line)
        # Remove space before punctuation
        normalized_line_str = re.sub(r'\s+([,.!?])', r'\1', normalized_line_str)
        return normalized_line_str

    # Split text into lines
    lines = text.splitlines()
    
    # Normalize each line individually
    normalized_lines = [normalize_line(line) for line in lines]
    
    # Join lines back with the original line breaks
    return '\n'.join(normalized_lines)

def convert_to_lower_except_first(text):
    patterns = ['DET', 'CAPT', 'OFC', 'SGT', 'MR', 'DR', 'SUPV', 'LT', 'TD', "MGR", "PO", "SIS", "SLD"]
    pattern_regex = r'\b(?:{})\b'.format('|'.join(patterns))

    def replace(match):
        return match.group(0)[0] + match.group(0)[1:].lower()

    result = re.sub(pattern_regex, replace, text)
    return result

def convert_specific_strings_to_lower(text):
    # Define the strings to convert to lowercase
    strings_to_convert = [
        "Victim", "OPERATOR", "CONDUCTOR", "ENGINEER", "OFFICER", "ASSAULT",
        "SUPERINTENDENT TRANSPORTATION", "TRACK WALKER", "SUPERVISOR","VAN",
        "SUPERINTENDENT OF TRANSPORTATION", "CHIEF OFFICER OF OPERATIONS",
        "SUPERINTENDENT", "DEPUTY SUPERINTENDENT","SYSTEM SAFETY", "CAPTAIN",
        "SUPERINTENDENT OF INFRASTRUCTURE", "SUPERVISOR OF INFRASTRUCTURE",
        "TRACK FOREMAN", "SUPERINTENDENT OF MAINTENANCE OF WAY", "CHIEF OFFICER MAINTAINANCE OF WAY"
    ]
    
    # Create a regex pattern to match the exact strings
    pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, strings_to_convert)) + r')\b', re.IGNORECASE)
    
    # Convert matched strings to lowercase
    def replace(match):
        return match.group(0).lower()
    
    return pattern.sub(replace, text)


# List of patterns to match
patterns_lowercase_no_space = {
    'ac', 'ad', 'af', 'ao', 'bb', 'bc', 'bd', 'bf', 'bh', 'bi', 'bm', 'bo', 'bq', 'bt', 'bw',
    'ca', 'ce', 'cm', 'co', 'cr', 'cs', 'cv', 'cw', 'da', 'db', 'dn', 'dr', 'ds', 'ea', 'eb', 'ef',
    'ej', 'el', 'eo', 'er', 'es', 'ew', 'fa', 'fb', 'fg', 'fh', 'fi', 'fm', 'fn', 'fo', 'fp', 'fr',
    'fs', 'fu', 'fw', 'gs', 'ha', 'hf', 'hm', 'hr', 'hw', 'ib', 'io', 'jz', 'kl', 'kt',
    'la', 'lc', 'lf', 'lr', 'ls', 'ma', 'mb', 'mc', 'mh', 'mn', 'mo', 'mr', 'ms', 'mt', 'mu', 'mv',
    'mw', 'na', 'nb', 'nd', 'ne', 'nl', 'nn', 'nq', 'nr', 'ns', 'nw', 'ob', 'od', 'op', 'ov',
    'pa', 'po', 'ps', 'pu', 'pv', 'qb', 'qn', 'qw', 'ra', 'rc', 're', 'rf', 'ro', 'rr', 'rs', 'rt',
    'rw', 'sa', 'sb', 'sc', 'sd', 'se', 'sh', 'si', 'sm', 'sn', 'so', 'sp', 'ss', 'sv', 'sw', 'ta',
    'tc', 'td', 'tf', 'tp', 'tr', 'ts', 'tw', 'uc', 'vc', 'vo', 'wa', 'wb', 'wc', 'wf', 'wh',
    'wm', 'wo', 'ws', 'wt', 'ww', 'xo', 'ym', 'yo'
}

# Create a dictionary for reverting patterns to uppercase
reverse_pattern_dict = {pattern: pattern.upper() for pattern in patterns_lowercase_no_space}

# Create a regex pattern that matches any of the lowercase patterns
patterns_regex = r'\b(?:' + '|'.join(re.escape(pattern) for pattern in patterns_lowercase_no_space) + r')\b'

def revert_patterns(text):
    """
    Reverts matched patterns in the text to their uppercase form.

    Parameters:
    - text: The input text containing patterns to be reverted.

    Returns:
    - The text with patterns reverted to uppercase.
    """
    def replace_match(match):
        matched_pattern = match.group(0)
        return reverse_pattern_dict.get(matched_pattern, matched_pattern)
    
    # Use re.sub with a function to replace matches
    return re.sub(patterns_regex, replace_match, text)

# Function to process and scrub text
def process_and_scrub_text(text):
    
    text = text.replace("  ", " ")  # Replace double spaces with single space
    text = text.replace('#', '')
    text = re.sub(r'\b([a-zA-Z])\/([a-zA-Z])\b', lambda m: (m.group(1) + m.group(2)).lower(), text)
    # Transform specific patterns to lowercase except for the first letter
    text = convert_to_lower_except_first(text)
    text = re.sub(r'\(([^)]+)\)', lambda m: f"({ ' '.join(word.capitalize() if len(word) > 2 else word for word in m.group(1).split()) })", text)
    text = re.sub(r'\bDet\b(?!\.)', 'detective', text)
    text = text.replace('P.O.', 'Officer').replace("Sgt.", "Sgt").replace("Supv.", "supervisor")
    text= convert_specific_strings_to_lower(text)
    text = re.sub('(?<=[a-z])(?=[A-Z])', ' ', text) #
    text = normalize_text(text)
    text = scrub_vehicle_numbers(text) # Call function to scrub vehicle numbers
    text = text.replace("  ", " ")
    
   
    try: # Call PII to scrub sensitive info within the text column
        analysis_results = pii_analysis_service.analyze_collection([text])
        results = analysis_results.to_dict()
        sanitized_texts = [result["sanitized_text"] for result in results["analyses"]]
        scrubbed_text_ = sanitized_texts[0]
        scrubbed_text_= scrub_names_after_occupation_title(sanitized_texts[0])
        scrubbed_text_= handle_pii_exception(scrubbed_text_) 
        
        
    except Exception as e:  # This will handle PII exceptions if any and print an error message 
        print(f"An error occurred while processing the text: {text}. Error: {e}")
        scrubbed_text_ = handle_pii_exception(text)
     
    
    
    return scrubbed_text_  

start = time.time()  # Start time for overall process


# Create an empty list to store the scrubbed texts
scrubbed_texts = []

with tqdm(total=len(df2)) as pbar:
    for _, row in df2.iterrows():
        scrubbed_text = process_and_scrub_text(row[ 'Event Description'])
        scrubbed_texts.append(scrubbed_text)
        pbar.update(1)  # Update progress bar

end = time.time()  # End time for overall process
print(f"Total processing time: {end - start} seconds")


# Replace the 'Incident_Description' column with the scrubbed texts
df2['Incident_Desc_Scrubbed'] = scrubbed_texts
df2['Incident_Desc_Scrubbed'] = df2['Incident_Desc_Scrubbed'].apply(revert_patterns)

#Rename 'Incident_Desc_Scrubbed' column
df2.drop(columns=['Event Description'], inplace=True)
df2.rename({'Incident_Desc_Scrubbed': 'Event Description'}, axis=1, inplace=True)
df2.insert(11, 'Event Description', df2.pop('Event Description'))


# Save the modified DataFrame to a new CSV file
df2.to_csv('Scrubbed_S&S_August2024_test.csv', index= False)

100%|██████████| 10/10 [00:01<00:00,  7.53it/s]


Total processing time: 1.3433198928833008 seconds
